# Imports

In [1]:
!pwd
import sys
from bumps.names import Curve, fit, FitProblem
from bumps.dream.state import load_state
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as fft
import os

sys.path.append("../../../..")
import molgroups as mol

sys.path.append("../../../../PlottingAndAnalysis")
import rs
import rsdi

sys.path.append("../../../../information_theory")
import entropy

/Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/information_theory/entropy_gridsearch_singlethread/garefl


# Create Short Example Fit

In [2]:
!refl1d run.py --fit=dream --store=T --init=lhs --parallel --burn=100 --steps=5 --overwrite

zsh:1: /usr/local/bin//refl1d: bad interpreter: /usr/local/opt/python@3.8/bin/python3.8: no such file or directory
# /Users/frank/opt/anaconda3/envs/Python37/bin/refl1d run.py --fit=dream --store=T --init=lhs --parallel --burn=100 --steps=5 --overwrite --seed=450457
[0] = l_siox = 18 in [10,30]
[1] = rho_siox = 3.55 in [3.2,3.8]
[2] = l_submembrane = 2.5 in [1,10]
[3] = l_lipid1 = 11.8 in [10,15]
[4] = l_lipid2 = 12.4 in [10,15]
[5] = vf_bilayer = 0.95 in [0.9,1]
[6] = rho_solv_0 = -0.5 in [-1.04,-0.44]
[7] = global_rough = 3 in [2,5]
[8] = sigma = 2.5 in [2,5]
[9] = background_0 = -8 in [-9,-5]

[chisq=264.32(17), nllf=8854.56]
# steps: 5, # draws: 500
step 1 cost 264.32(17)
                                  l_siox ...|......         18 in (10,30)
                                rho_siox .....|....       3.55 in (3.2,3.8)
                           l_submembrane .|........        2.5 in (1,10)
                                l_lipid1 ...|......       11.8 in (10,15)
                  

# Variables

In [3]:
# general fit setup
setupdir = os.getcwd()
runfile = "run"
store = 'T'
fitsource = "garefl"

# particular entropy setup
burn = 8000
steps = 500
convergence = 2.0
miniter = 1
mode = 'water'
bClusterMode = False
bFetchMode = False
time = 2
bcalcsymmetric = True
upper_info_plotlevel = None
plotlimits_filename = " "
calcsingle = False

# setup batchscript for SLURM (if used)
script = []
script.append('#!/bin/bash\n')
script.append('#SBATCH --job-name=entro {mcmc_iteration}\n')
script.append('#SBATCH -A mc4s9np\n')
script.append('#SBATCH -p RM\n')
script.append('#SBATCH -t 0' + str(time) + ':00:00\n')
script.append('#SBATCH -N 4\n')
script.append('#SBATCH --ntasks-per-node 28\n')
script.append('\n')
script.append('set +x\n')
script.append('cd $SLURM_SUBMIT_DIR\n')
# script.append('cd '+dirname+'\n')
script.append('\n')
script.append('module load python/2.7.11_gcc\n')
script.append('export PYTHONPATH=/home/hoogerhe/bin/lib/python2.7/site-packages:/home/hoogerhe/src/bumps\n')
script.append('\n')
script.append('mpirun -np 112 python /home/hoogerhe/src/refl1d/bin/refl1d_cli.py {mcmc_dirname}/run.py --fit=dream --mpi --init=lhs --batch --pop=28 --time=' 
              + str(float(time) - 0.1) + ' --thin=20 --store={mcmc_dirname}/save --burn=' + str(burn) 
              + ' --steps=' + str(steps) + '\n')


# Fit Setup

In [4]:
entr = entropy.Entropy(
    fitsource=fitsource,
    spath=setupdir,
    mcmcpath=store,
    runfile=runfile,
    mcmcburn=burn, 
    mcmcsteps=steps, 
    convergence=convergence, 
    miniter=miniter, 
    mode=mode,                      
    bClusterMode=bClusterMode, 
    bFetchMode=bFetchMode, 
    calc_symmetric=bcalcsymmetric,
    upper_info_plotlevel=upper_info_plotlevel, 
    plotlimits_filename=plotlimits_filename,
    slurmscript=script
)

g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o


setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.


cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/boxmin -lboxmin  -lm 
g++ -shared -o model.so setup.o /Users/frank/danse/refl1d/garefl/src/refl1d.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl  -lm 


In [ ]:
entr.run_optimization()

No file: /Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/information_theory/entropy_gridsearch_singlethread/garefl/iteration_0/save/run.py
No state to reload.
No problem to reload.
Found T/run.par 

Loading MCMC best-fit parameters ...
l_siox 18.0
rho_siox 3.55e-06
l_submembrane 2.5
l_lipid1 11.8
l_lipid2 12.4
vf_bilayer 0.95
rho_solv_0 -5e-07
global_rough 3.0
sigma 2.5
background_0 -8.0
g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o


setup.cc:103:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.


cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/boxmin -lboxmin  -lm 
g++ -shared -o model.so setup.o /Users/frank/danse/refl1d/garefl/src/refl1d.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl  -lm 
chisq = 264.315


rm: /Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/information_theory/entropy_gridsearch_singlethread/garefl/iteration_0/save: No such file or directory


g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o


setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.


cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/boxmin -lboxmin  -lm 
ln -s /Users/frank/danse/refl1d/garefl/src/gaplot gaplot
g++ -shared -o model.so setup.o /Users/frank/danse/refl1d/garefl/src/refl1d.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl  -lm 


In [ ]:
import pandas

fig, ax = plt.subplots()

i = 0
while os.path.isfile('sim' + str(i) + '.dat'):
    simdata = pandas.read_csv('sim' + str(i) + '.dat', sep=' ', skip_blank_lines=True,comment='#')
    ax.errorbar(simdata['Q'], simdata['R'], simdata['dR'], label="sim " + str(i))
    i += 1

ax.legend(fontsize=8)
ax.set_ylabel("R", fontsize=10)
ax.set_yscale('log')
ax.minorticks_on()
ax.tick_params(which="both", direction="in", labelsize=10)
ax.tick_params(bottom=True, top=True, left=True, right=True, which="both")
ax.set_xlabel("$q_z$ (Å$^{-1}$)", fontsize=10)
ax.figure.set_size_inches(12.5, 7.5)
plt.savefig(store + "/model_comparison.png", facecolor="white")
plt.show()
